In [ ]:
import csv
from pathlib import Path
data_path = Path('..') / 'quick_analysis_results.csv'
rows = list(csv.DictReader(open(data_path)))
speedups = [float(r['full_time'])/float(r['quick_time']) for r in rows]
circuits = [f"{r['qubits']}q" for r in rows]


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 4))
plt.bar(circuits, speedups)
plt.ylabel('Speedup (×)')
plt.xlabel('Circuit')
plt.title('Relative Speedups')
plt.show()
